# Chemical potential in 3D lattice

Contact yifei.guan@epfl.ch if you have any problems or feedbacks to this notebook

## Introduction

This notebook gives a simple illustration to the variation of chemical potential for Fermions in 3D, following the Fermi-Dirac statistics in finite temperature. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
from ipywidgets import interactive as jupyter_interactive
from ipywidgets import interact
from scipy import integrate

%matplotlib notebook

# Basic libraries

In 3D, we have the  density of states $\rho \propto \sqrt{E}$. At zero temperature, the distribution of electrons just follows $$N(E)=a \sqrt{E}$$ and the Fermi energy is determined by the total occupied number $N_{occ}$ :$E_f$ is the energy such that $$N_{occ}=\int_{0}^{E_f} N(E) dE$$
   
In finite temperature, however, one must also consider the Fermi-dirac distribution of $$f(E)=\frac{1}{e^{(E-\mu)/kbT}+1}$$ when calculating the total occupation number. The expression is then $$ N_{occ}=\int_{0}^{\infty} f(E)N(E) dE$$
  
The chemical potential $\mu$ must be consistent with the occupation number. As shown in the following, the chemical potential changes at finite temperature.

The above plots give a simple illustration of variation of the chemical potential at finite temperature. At finite temperature, the simple calculation with zero-temperature $E_f$ is no longer consistent with the occupation number. The calculation of the varied fermi energy is shown in chapter 2.2.3 with some simple approximation.  

Alternatively, we can also calculate the variation of chemical potential with numerical methods. In the content of numerical evaluation, the consistency problem of chemical potantial and the occupation number can be solved through iteration.

In [6]:
def interactive_chem_consist(kt,ef):
    '''
    :param k: wave number
    '''
    kBT = 0.001 # Boltzmann constant measured interms of chemical potential
    E_min = 0.0001
    E_max = 3.0*ef
    factor = 1.0
    N = 200
    N1=40
    # Set up the variable space for 1D plot
    En_mu = np.linspace(E_min, E_max, N) #Energy measured in units of chemical potential
    
    
   
    ef1=ef
    nocc=2.0/3.0*(ef**(3.0/2.0))
    
    def nfg(e0):
        v,err=scipy.integrate.quad(cf,0,e0)
        return v
    def fg(e0):
        f1=lambda x: np.sqrt(x)
        v,err=scipy.integrate.quad(f1,0,e0)
        return v
    
    num_s=np.linspace(1,N1,N1)
    ef1_s=np.linspace(ef,ef,N1)
    for num in range(1,N1):
        cf=lambda x:np.sqrt(x)/(np.exp( (factor*x-ef1)/kt )+1) 
        c_f = cf(En_mu)  #np.sqrt(En_mu)/(np.exp( (factor*En_mu-1)/kt )+1) # Occupation number according to F-D statistics
        c_df = np.gradient(c_f) # Take the first derivative of the F-D distribution.


        #print(nfg(0.500))
        n_e=(1.5*nfg(E_max))**(2.0/3.0)
        
        ef1_s[num]=ef1
        ef1=ef1-0.2*(n_e-ef)
        
    ff0=lambda x:1.0/(np.exp( (factor*x-ef)/kt )+1)
    cf0=lambda x:np.sqrt(x)/(np.exp( (factor*x-ef)/kt )+1) 
    ff=lambda x:1.0/(np.exp( (factor*x-ef1)/kt )+1)
    cf=lambda x:np.sqrt(x)/(np.exp( (factor*x-ef1)/kt )+1) 
    
    def nfg0(e0):
        v,err=scipy.integrate.quad(cf0,0,e0)
        return v
    
    #plt.ylim(0,1.2)
    print("the filling:",nocc)
    print ("The integration without fixing the chemical potential",nfg0(E_max))
    print("corrected chem_potential=",ef1)
    print("the filling with corrected chemical potential:",nfg(E_max))
    #print("var chem_potential %=",100*(ef1-ef)/ef,"%")
    print("(kt/ef)^2:",(kt/ef)**2)
    print("var chem_potential:",np.abs((ef1-ef)/ef))
    print("d(mu)/(kt/ef^2)",np.abs((ef1-ef)/ef)/(kt/ef)**2)
    plt.figure(figsize=(20,60))
    plt.subplot(312)
    plt.xlim=(5.0,40.0)
    plt.plot(num_s,ef1_s)
    plt.xlabel('Number of iterations')
    plt.title('corrected chemical potential')
    
    
    
    
    f0=ff0(En_mu)
    c_f0=cf0(En_mu)
    
    f =  ff(En_mu)# Occupation number according to F-D statistics
    df = np.gradient(f) # Take the first derivative of the F-D distribution.

    
    c_f = cf(En_mu)  #np.sqrt(En_mu)/(np.exp( (factor*En_mu-1)/kt )+1) # Occupation number according to F-D statistics
    c_df = np.gradient(c_f) # Take the first derivative of the F-D distribution.
    n_fg=[nfg(x) for x in En_mu]
    n_fg0=[nfg0(x) for x in En_mu]
    n_f=[fg(x) for x in En_mu]
    
    
    plt.subplot(311)
    #plt.ylim(0.4,0.8)
    plt.ylim(0.0,1.10)
    #plt.xlim(0.9,1.1)
    plt.plot(En_mu, f0,label='F-D distribution')
    plt.plot(En_mu, c_f0)
    plt.plot(En_mu,n_fg0)
    #plt.plot(En_mu,n_f)
    #plt.plot([0,E_max],[nfg(E_max),nfg(E_max)])
    #plt.plot([ef-0.2,ef+0.2],[0.5,0.5])
    plt.plot([0,E_max],[nocc,nocc],label='filling')
    plt.plot([ef,ef],[0.0,0.5],label='ef')
    plt.legend()
    plt.xlabel('Energy')
    plt.title('without the correction')
    
    plt.subplot(313)
    #plt.ylim(0.4,0.8)
    plt.ylim(0.0,1.10)
    #plt.xlim(0.9,1.1)
    plt.plot(En_mu, f)
    plt.plot(En_mu, c_f)
    plt.plot(En_mu,n_fg)
    #plt.plot(En_mu,n_f)
    #plt.plot([0,E_max],[nfg(E_max),nfg(E_max)])
    #plt.plot([ef1-0.2,ef1+0.2],[0.5,0.5])
    plt.plot([ef1,ef1],[0.0,0.5],label='mu')
    plt.plot([ef,ef],[0.0,0.5],label='ef')
    plt.plot([0,E_max],[nocc,nocc],label='filling')
    plt.legend()
    plt.xlabel('Energy')
    plt.title('with the correction')


In [7]:
interact(interactive_chem_consist,kt=(0.01,0.20,0.005),ef=(0.01,2.00,0.01))

interactive(children=(FloatSlider(value=0.105, description='kt', max=0.2, min=0.01, step=0.005), FloatSlider(v…

<function __main__.interactive_chem_consist(kt, ef)>

So far we have finished the iterative calculation of chemical potantial variation. However, it is worth looking into thechange of  $E_f$  in reality.  
The complete expression of electron density writes:$$ \rho(E)=\frac{1}{2\pi ^2}(\frac{2m}{\hbar ^2})^{3/2} \cdot E^{1/2} \times \frac{1}{\exp (\frac{E-\mu}{k_B T}) +1}$$  
Up to the first term, the chemical potential $$\mu=E_f \cdot [1-\frac{\pi ^2}{12}(\frac{k_B T}{E_f})^2]$$ changes from $E_f$ approximately with $(\frac{k_B T}{E_f})^2$. In reality, the $k_B T$ is around 25meV at room temperature(300K), while in metals the fermi energy is in the scale of several eVs($10^5 $K). Thus the term $(\frac{k_B T}{E_f})^2$ is in the magnitude of $~\frac{1}{10000}^2$ and the correction to chemical potential is not significant.
